In [1]:
import json
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Cargar datos desde JSON
with open('intents.json') as file:
    data = json.load(file)

# Preprocesamiento de datos
intents = data['intents']
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in intents:
    for pattern in intent['patterns']:
        word_list = pattern.split()
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([w.lower() for w in words if w not in ignore_words]))
classes = sorted(set(classes))

# Preparar datos de entrenamiento
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [w.lower() for w in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Asegurarse de que los arrays tengan la misma longitud
train_x = []
train_y = []

for feature, label in training:
    train_x.append(feature)
    train_y.append(label)

train_x = np.array(train_x)
train_y = np.array(train_y)

# Crear el modelo
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Guardar el modelo
model.save('chatbot_model.h5')


Epoch 1/200
52/52 [==============================] - 1s 1ms/step - loss: 2.1199 - accuracy: 0.2423
Epoch 2/200
52/52 [==============================] - 0s 1ms/step - loss: 1.8610 - accuracy: 0.3808
Epoch 3/200
52/52 [==============================] - 0s 1ms/step - loss: 1.6587 - accuracy: 0.3654
Epoch 4/200
52/52 [==============================] - 0s 1ms/step - loss: 1.4983 - accuracy: 0.4308
Epoch 5/200
52/52 [==============================] - 0s 1ms/step - loss: 1.3278 - accuracy: 0.5462
Epoch 6/200
52/52 [==============================] - 0s 1ms/step - loss: 1.1871 - accuracy: 0.6308
Epoch 7/200
52/52 [==============================] - 0s 1ms/step - loss: 1.0288 - accuracy: 0.7154
Epoch 8/200
52/52 [==============================] - 0s 1ms/step - loss: 0.8891 - accuracy: 0.7346
Epoch 9/200
52/52 [==============================] - 0s 1ms/step - loss: 0.8036 - accuracy: 0.7808
Epoch 10/200
52/52 [==============================] - 0s 1ms/step - loss: 0.6828 - accuracy: 0.8038
Epoch 11/

c:\Users\libra\Documents\chatbot\chatbot_env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
import json
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD

# Función para cargar datos desde JSON
def load_data():
    with open('intents.json', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Función para guardar datos en el archivo JSON
def save_data(data, data_file='intents.json'):
    with open(data_file, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)

data = load_data()

# Inicializar el modelo
model = None

# Preprocesamiento de datos
ignore_words = ['?', '!', '.', ',']

def preprocess_data(data):
    intents = data['intents']
    words = []
    classes = []

    for intent in intents:
        for pattern in intent['patterns']:
            word_list = pattern.split()
            words.extend([w.lower() for w in word_list if w not in ignore_words])
        classes.append(intent['tag'])

    words = sorted(set(words))
    classes = sorted(set(classes))
    return words, classes

def clean_up_sentence(sentence):
    sentence_words = sentence.split()
    sentence_words = [word.lower() for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model, words, classes):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

def add_new_intent(user_input, user_response):
    data = load_data()
    new_intent = {
        "tag": "nueva_intencion",
        "patterns": [user_input],
        "responses": [user_response]
    }
    data['intents'].append(new_intent)
    save_data(data)

def retrain_model():
    global model
    data = load_data()
    words, classes = preprocess_data(data)
    documents = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            word_list = pattern.split()
            documents.append((word_list, intent['tag']))

    training = []
    output_empty = [0] * len(classes)

    for doc in documents:
        bag = []
        pattern_words = doc[0]
        pattern_words = [word.lower() for word in pattern_words]
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)

        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training.append([bag, output_row])

    random.shuffle(training)
    training = np.array(training, dtype=object)  # Asegurarse de que todos los elementos tengan la misma longitud

    train_x = np.array(training[:, 0].tolist())
    train_y = np.array(training[:, 1].tolist())

    model = Sequential()
    model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(train_y[0]), activation='softmax'))

    sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)  # Usando el optimizador de legado
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
    model.save('chatbot_model.h5')
    print("Modelo reentrenado y guardado!")

# Proceso de reentrenamiento al inicio de cada interacción
retrain_model()
words, classes = preprocess_data(data)

print("¡Chatbot listo para conversar!")

while True:
    message = input("")
    ints = predict_class(message, model, words, classes)
    
    # Salir si la intención es de despedida
    if ints[0]['intent'] == "despedida":
        res = get_response(ints, data)
        print(f'>> {message}')
        print(f">> {res}")
        break

    res = get_response(ints, data)
    print(f'>> {message}')
    print(f'>> {res}')

    # Guardar nuevas entradas y respuestas automáticamente
    print("Por favor ingresa la respuesta correcta del chatbot:")
    user_response = input("")
    add_new_intent(message, user_response)
    retrain_model()
    words, classes = preprocess_data(load_data())


Epoch 1/200
52/52 [==============================] - 0s 1ms/step - loss: 1.9939 - accuracy: 0.3077
Epoch 2/200
52/52 [==============================] - 0s 1ms/step - loss: 1.6380 - accuracy: 0.3808
Epoch 3/200
52/52 [==============================] - 0s 1ms/step - loss: 1.5221 - accuracy: 0.4885
Epoch 4/200
52/52 [==============================] - 0s 1ms/step - loss: 1.3183 - accuracy: 0.5385
Epoch 5/200
52/52 [==============================] - 0s 983us/step - loss: 1.1398 - accuracy: 0.6308
Epoch 6/200
52/52 [==============================] - 0s 1000us/step - loss: 0.9621 - accuracy: 0.7038
Epoch 7/200
52/52 [==============================] - 0s 3ms/step - loss: 0.7983 - accuracy: 0.7231
Epoch 8/200
52/52 [==============================] - 0s 1ms/step - loss: 0.6711 - accuracy: 0.7423
Epoch 9/200
52/52 [==============================] - 0s 1000us/step - loss: 0.6183 - accuracy: 0.7846
Epoch 10/200
52/52 [==============================] - 0s 1ms/step - loss: 0.6104 - accuracy: 0.8000
E

c:\Users\libra\Documents\chatbot\chatbot_env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modelo reentrenado y guardado!
¡Chatbot listo para conversar!
1/1 [==============================] - 0s 48ms/step
>> hola
>> ¡Hola! ¿Cómo te sientes?
Por favor ingresa la respuesta correcta del chatbot:
Epoch 1/200
53/53 [==============================] - 0s 1ms/step - loss: 2.0669 - accuracy: 0.2989
Epoch 2/200
53/53 [==============================] - 0s 1ms/step - loss: 1.7661 - accuracy: 0.3755
Epoch 3/200
53/53 [==============================] - 0s 1ms/step - loss: 1.5466 - accuracy: 0.4674
Epoch 4/200
53/53 [==============================] - 0s 1ms/step - loss: 1.2564 - accuracy: 0.6092
Epoch 5/200
53/53 [==============================] - 0s 1ms/step - loss: 1.0679 - accuracy: 0.6437
Epoch 6/200
53/53 [==============================] - 0s 1ms/step - loss: 0.9569 - accuracy: 0.6897
Epoch 7/200
53/53 [==============================] - 0s 1ms/step - loss: 0.7881 - accuracy: 0.7510
Epoch 8/200
53/53 [==============================] - 0s 1ms/step - loss: 0.7856 - accuracy: 0.7241
Epoch